#### **This Notebook is to send this data to big query warehouse made from the Models Cars Cleaned Data **

#### **Install G cloud libraries**

In [0]:
%pip install google-cloud-bigquery pandas-gbq

  Obtaining dependency information for google-cloud-bigquery from https://files.pythonhosted.org/packages/2a/91/e1c80ae2924efc047ca156662d6b0458d9a9ce99204ae7e719ff9a66123d/google_cloud_bigquery-3.26.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pandas-gbq from https://files.pythonhosted.org/packages/a9/eb/f35416f452de18575b216aaf4b4710beb7201dce4e07265c5134e1d1ecfd/pandas_gbq-0.23.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for db-dtypes<2.0.0,>=1.0.4 from https://files.pythonhosted.org/packages/89/d0/f746b7f88e3f6e6c6753fb7a078d7d2db2b8f969c7913d51b3a54f0abe53/db_dtypes-1.3.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pydata-google-auth>=1.5.0 from https://files.pythonhosted.org/packages/28/6b/3320c9ddbfc572108917e8432a07e8bd1e40054d94b5ad40c755afdc1160/pydata_google_auth-1.8.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/239.1 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
from pyspark.sql import SparkSession
import pandas as pd
from google.cloud import bigquery
import pandas_gbq
from google.oauth2 import service_account

#### **Spark Write to Big Query**


In [0]:

# Set your GCP project and dataset
project_id = "shining-bazaar-436810-p2"
dataset_id = "model_cars"

# Path to your service account key in DBFS (adjust to your file's location)
gcp_credentials_path = "xrbams_key.json"

# Authenticate using credentials
credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)

spark = SparkSession.builder.appName("BigQueryUploader").getOrCreate()

# Dictionary mapping CSV filenames to BigQuery table names
csv_to_bq_table = {
    "cleaned_customers.csv": "customers",
    "cleaned_employees.csv": "employees",
    "cleaned_offices.csv": "offices",
    "cleaned_orderdetails.csv": "orderdetails",
    "cleaned_orders.csv": "orders",
    "cleaned_payments.csv": "payments",
    "cleaned_productlines.csv": "productlines",
    "cleaned_products.csv": "products",
    "cleaned_warehouses.csv": "warehouses"
}

# Iterate over the files and corresponding BigQuery tables
for csv_file, table_name in csv_to_bq_table.items():
    file_path = f"dbfs:/tmp/{csv_file}"
    
    # Read each CSV file into a Spark DataFrame
    df = spark.read.option("header", "true").csv(file_path)
    
    # Show the data (for debugging purposes)
    print(f"Loaded data from {csv_file}:")
    df.show(5)  # Show the first 5 rows
    
    # Convert Spark DataFrame to Pandas DataFrame
    pdf = df.toPandas()
    
    # Define the full table name for BigQuery
    full_table_name = f"{project_id}.{dataset_id}.{table_name}"
    
    # Upload data to BigQuery using pandas-gbq
    print(f"Uploading data to BigQuery table: {full_table_name}")
    pandas_gbq.to_gbq(
        pdf, 
        full_table_name, 
        project_id=project_id, 
        credentials=credentials, 
        if_exists='replace'  # Use 'append' if you don't want to overwrite existing data
    )

    print(f"Data from {csv_file} uploaded successfully to {full_table_name}")

Loaded data from cleaned_customers.csv:
+--------------+--------------------+---------------+----------------+------------+--------------------+------------+---------+--------+----------+---------+----------------------+-----------+
|customerNumber|        customerName|contactLastName|contactFirstName|       phone|        addressLine1|addressLine2|     city|   state|postalCode|  country|salesRepEmployeeNumber|creditLimit|
+--------------+--------------------+---------------+----------------+------------+--------------------+------------+---------+--------+----------+---------+----------------------+-----------+
|           103|   Atelier graphique|        Schmitt|         Carine |  40.32.2555|      54, rue Royale|        NULL|   Nantes|    None|     44000|   France|                1370.0|    21000.0|
|           112|  Signal Gift Stores|           King|            Jean|  7025551838|     8489 Strong St.|        NULL|Las Vegas|      NV|     83030|      USA|                1166.0|    7180

100%|██████████| 1/1 [00:00<00:00, 11848.32it/s]


Data from cleaned_customers.csv uploaded successfully to shining-bazaar-436810-p2.model_cars.customers
Loaded data from cleaned_employees.csv:
+--------------+---------+---------+---------+--------------------+----------+---------+--------------------+
|employeeNumber| lastName|firstName|extension|               email|officeCode|reportsTo|            jobTitle|
+--------------+---------+---------+---------+--------------------+----------+---------+--------------------+
|          1002|   Murphy|    Diane|    x5800|dmurphy@classicmo...|         1|     NULL|           President|
|          1056|Patterson|     Mary|    x4611|mpatterso@classic...|         1|   1002.0|            VP Sales|
|          1076| Firrelli|     Jeff|    x9273|jfirrelli@classic...|         1|   1002.0|        VP Marketing|
|          1088|Patterson|  William|    x4871|wpatterson@classi...|         6|   1056.0|Sales Manager (APAC)|
|          1102|   Bondur|   Gerard|    x5408|gbondur@classicmo...|         4|   1056.0

100%|██████████| 1/1 [00:00<00:00, 10565.00it/s]


Data from cleaned_employees.csv uploaded successfully to shining-bazaar-436810-p2.model_cars.employees
Loaded data from cleaned_offices.csv:
+----------+-------------+---------------+--------------------+------------+----------+-------+----------+---------+
|officeCode|         city|          phone|        addressLine1|addressLine2|     state|country|postalCode|territory|
+----------+-------------+---------------+--------------------+------------+----------+-------+----------+---------+
|         1|San Francisco|+1 650 219 4782|   100 Market Street|   Suite 300|        CA|    USA|     94080|       NA|
|         2|       Boston|+1 215 837 0825|    1550 Court Place|   Suite 102|        MA|    USA|     02107|       NA|
|         3|          NYC|+1 212 555 3000|523 East 53rd Street|     apt. 5A|        NY|    USA|     10022|       NA|
|         4|        Paris|+33 14 723 4404|43 Rue Jouffroy D...|        NULL|      None| France|     75017|     EMEA|
|         5|        Tokyo|+81 33 224 500

...

Data from cleaned_productlines.csv uploaded successfully to shining-bazaar-436810-p2.model_cars.productlines
Loaded data from cleaned_products.csv:
+-----------+--------------------+------------+------------+--------------------+--------------------+---------------+-------------+--------+------+
|productCode|         productName| productLine|productScale|       productVendor|  productDescription|quantityInStock|warehouseCode|buyPrice|  MSRP|
+-----------+--------------------+------------+------------+--------------------+--------------------+---------------+-------------+--------+------+
|   S10_1678|1969 Harley David...| Motorcycles|        1:10|     Min Lin Diecast|This replica feat...|           7933|            a|   48.81| 95.70|
|   S10_1949|1952 Alpine Renau...|Classic Cars|        1:10|Classic Metal Cre...|Turnable front wh...|           7305|            b|   98.58|214.30|
|   S10_2016|1996 Moto Guzzi 1...| Motorcycles|        1:10|Highway 66 Mini C...|Official Moto Guz...|     

100%|██████████| 1/1 [00:00<00:00, 9489.38it/s]


Data from cleaned_products.csv uploaded successfully to shining-bazaar-436810-p2.model_cars.products
Loaded data from cleaned_warehouses.csv:
+-------------+-------------+---------------+
|warehouseCode|warehouseName|warehousePctCap|
+-------------+-------------+---------------+
|            a|        North|             72|
|            b|         East|             67|
|            c|         West|             50|
|            d|        South|             75|
+-------------+-------------+---------------+

Uploading data to BigQuery table: shining-bazaar-436810-p2.model_cars.warehouses


100%|██████████| 1/1 [00:00<00:00, 11983.73it/s]

Data from cleaned_warehouses.csv uploaded successfully to shining-bazaar-436810-p2.model_cars.warehouses
